# Reading CSV database

**1. Set `runtime type` to `GPU`.**

In [1]:
!git clone https://github.com/LeSnakk/TextReadabilityAnnotation

Cloning into 'TextReadabilityAnnotation'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 68 (delta 15), reused 54 (delta 10), pack-reused 0
Receiving objects: 100% (68/68), 16.18 MiB | 28.97 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
import pandas as pd

In [11]:
database_path = '/kaggle/working/TextReadabilityAnnotation/project-files/annotation-data-text/text-preprocessed/CLEAR_Corpus_6.01_shortened.csv'
output_path = '/kaggle/working/TextReadabilityAnnotation/project-files/llm-data/llm-results/CLEAR_Corpus_6.01_LLM-output.csv'

In [ ]:
import shutil
shutil.copyfile(database_path, output_path)

In [12]:
# Extract excerpts from database
data = pd.read_csv(output_path)

excerpts = data['Excerpt'].tolist()

excerpts = [excerpt.replace('\n', ' ') for excerpt in excerpts]

print(f'Loaded database with {len(excerpts)} excerpts.')

Loaded database with 4724 excerpts.


# Setting up LLAMA model

In [5]:
!pip install transformers torch accelerate

In [ ]:
# Input HuggingFace token
# Make sure access to Llama 2 is granted
print('Please input your HuggingFace token:')
token = input()

!huggingface-cli login --token {token}

print('Logged in as:')
!huggingface-cli whoami

In [7]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-hf
# model = "meta-llama/Llama-2-13b-chat-hf" # meta-llama/Llama-2-13b-hf

tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:690: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [8]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [10]:
def get_llama_response(prompt: str) -> None:

    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=40,
        top_p=0.1,
        temperature=0.7,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=512,
    )
    generated_text = sequences[0]['generated_text']
    prompt_length = len(prompt)
    cleaned_text = generated_text[prompt_length:].strip()
    # print("Chatbot:", sequences[0]['generated_text'])
    return cleaned_text

In [20]:
prompt = 'To which model am I talking to?'
print(get_llama_response(prompt))

Answer: You are talking to the BERT model. BERT (Bidirectional Encoder Representations from Transformers) is a language model developed by Google in 2018. It is a pre-trained deep learning model that can be fine-tuned for a wide range of natural language processing (NLP) tasks, such as sentiment analysis, question-answering, and text classification. BERT is particularly useful for tasks that require a deep understanding of the nuances of language, such as language translation, summarization, and generation.


# Tasking the model

In [ ]:
prompt = f'What is the following text about: "{excerpts[8]}"?'
print(get_llama_response(prompt))

In [ ]:
prompt = f'Can you give me a readability score between 0, meaning hard to read, and 1, meaning easy to read "{excerpts[7]}"?'
print(get_llama_response(prompt))

In [ ]:
prompt = f'How would you rate the readability of the following text on a scale from 1, which means hard, to 10, which means easy "{excerpts[7]}"? Your answer should look like this "Score", following the score, and "Explanation", why you rated it like this.'
print(get_llama_response(prompt))

In [ ]:
for i in range(11):
  prompt = f'How would you rate the readability of the following text on a scale from 0, which means hard, to 100, which means easy "{excerpts[i]}"? Your answer should look like this "Score", following the score, and "Explanation", why you rated it like this.'
  print(f'Text {i}:')
  print(get_llama_response(prompt))

In [19]:
# Test model
for i in range(11):
  prompt = f'How would you rate the readability in percent of the following text 0% means hard to read, 100% means easy to read "{excerpts[i]}"? Your answer should look like this "Score", following the score, and "Explanation", why you rated it like this.'
  print(f'Text {i}:')
  response = get_llama_response(prompt)
  print(response)
  try:
    split_output = response.split('\n')
    if len(split_output) >= 2 and split_output[0].startswith("Score:") and split_output[1].startswith("Explanation:"):
        score_with_percentage = split_output[0].split(': ')[1]
        score = score_with_percentage.replace('%', '')
        explanation = split_output[1].split(': ')[1]
        print('SCORE: ', score)
        print('EXPLANATION: ', explanation)
    else:
        print("Output format not recognized.")
  except Exception as e:
    print("An error occurred:", str(e))

Text 0:
Score: 60%
Explanation: The text is written in a descriptive style, with vivid imagery and figurative language, which makes it easy to visualize the winter landscape. The use of words like "rumpled," "powdered," and "glittering" adds to the readability of the text. However, there are some complex sentences and long phrases that may make it difficult for some readers to follow. Overall, the text is moderately easy to read, with a readability score of around 60%.
SCORE:  60
EXPLANATION:  The text is written in a descriptive style, with vivid imagery and figurative language, which makes it easy to visualize the winter landscape. The use of words like "rumpled," "powdered," and "glittering" adds to the readability of the text. However, there are some complex sentences and long phrases that may make it difficult for some readers to follow. Overall, the text is moderately easy to read, with a readability score of around 60%.
Text 1:
For example: "Score: 60%, Explanation: The text is 

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Answer:

Score: 70%
Explanation: The text has a moderate level of readability, with a mix of complex and simple sentences. The use of compound sentences and adjectives like "stiff with gold" and "sculptured goblet" create some difficulty, but the overall structure is clear and easy to follow. The text also uses repetition, with the phrase "the heavens had given" appearing twice, which can make it feel more readable. However, some of the vocabulary, such as "walled round" and "pledged the merchant kings," may be unfamiliar to some readers, which could affect readability.
Unexpected output format. Unable to extract score and explanation.
Text 5:
Answer:
Score: 80%
Explanation: The text is written in a simple and clear manner, with short sentences and basic vocabulary. It is easy to follow the story and understand the characters and their belongings. However, some of the sentences are a bit long and could be broken up for better readability. Overall, the text is easy to read but could ben

KeyboardInterrupt: 

In [ ]:
# Add LLM columns to CSV data
new_column = {'LLAMA Score': '', 'LLAMA Explanation': ''}

data = pd.read_csv(output_path)

data['LLAMA Score'] = new_column['LLAMA Score']
data['LLAMA Explanation'] = new_column['LLAMA Explanation']

data.to_csv(output_path, index=False)

In [16]:
# Retrieve LLM score and explanation and add them to CSV data
for i in range(0, min(5000, len(excerpts))):    
  prompt = f'How would you rate the readability in percent of the following text 0% means hard to read, 100% means easy to read "{excerpts[i]}"? Your answer should look like this "Score", following the score, and "Explanation", why you rated it like this.'
  print(f'Text {i}:')
  response = get_llama_response(prompt)
  print(response)
  try:
    split_output = response.split('\n')
    if len(split_output) >= 2 and split_output[0].startswith("Score:") and split_output[1].startswith("Explanation:"):
        score_with_percentage = split_output[0].split(': ')[1]
        score = score_with_percentage.replace('%', '')
        explanation = split_output[1].split(': ')[1]
        print('SCORE: ', score)
        data.at[i, 'LLAMA Score'] = score
        print('EXPLANATION: ', explanation)
        data.at[i, 'LLAMA Explanation'] = explanation
        data.to_csv(output_path, index=False)
    else:
        print("Output format not recognized.")
        data.at[i, 'LLAMA Explanation'] = response
        data.to_csv(output_path, index=False)
  except Exception as e:
    print("An error occurred:", str(e))
    print(f'Text {i}:')

Text 4000:
For example, "Score: 80%, Explanation: The text is written in clear and concise language, with short sentences and simple vocabulary, making it easy to read.". Please answer with a score and an explanation for each text you rate.
Output format not recognized.
Text 4001:
0% - 20% = Very Hard to Read, 21% - 40% = Hard to Read, 41% - 60% = Somewhat Difficult, 61% - 80% = Easy to Read, 81% - 100% = Very Easy to Read.
Output format not recognized.
Text 4002:
For example, "Score: 80%, Explanation: The text is written in a clear and concise manner, with simple vocabulary and sentence structure, making it easy to read."
Output format not recognized.
Text 4003:
For example, "Score: 70%, Explanation: The text uses complex vocabulary and sentence structures, making it difficult to read for a beginner.".
Output format not recognized.
Text 4004:
Answer:
Score: 70%
Explanation: The text has a moderate level of readability, with a mix of simple and complex vocabulary, and a clear but somew

/tmp/ipykernel_42/2434353725.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '80' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.at[i, 'LLAMA Score'] = score


Text 4006:
For example:

Score: 60%
Explanation: The text is written in a clear and concise manner, with short sentences and simple vocabulary. However, it may be challenging for some readers to follow the narrative due to the lack of context and character development.

Please provide your answer in the format above.
Output format not recognized.
Text 4007:
0% - Very hard to read, 10% - Hard to read, 20% - Somewhat difficult, 30% - Neutral, 40% - Somewhat easy, 50% - Easy to read, 60% - Very easy to read, 70% - Extremely easy to read, 80% - Nearly effortless to read, 90% - Extremely easy to read, 100% - Nearly effortless to read.
Output format not recognized.
Text 4008:
For example:

Score: 80%
Explanation: This text is relatively easy to read, with a moderate level of complexity. The language is sophisticated, but the sentences are well-structured and easy to follow. The use of figurative language, such as "purest realization" and "radiant glory," adds depth and interest to the text w

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


For example, "Score: 60%, Explanation: The text is written in a clear and concise manner, with short sentences and simple vocabulary, making it easy to read."
Output format not recognized.
Text 4011:
Score: 80%
Explanation: The text is written in a clear and concise manner, with short sentences and simple vocabulary. The use of concrete nouns and verbs helps to create a vivid image of the scene, making it easy to visualize. However, some of the sentences are quite long and could be broken up for easier reading. Overall, the readability is high, but not perfect.
SCORE:  80
EXPLANATION:  The text is written in a clear and concise manner, with short sentences and simple vocabulary. The use of concrete nouns and verbs helps to create a vivid image of the scene, making it easy to visualize. However, some of the sentences are quite long and could be broken up for easier reading. Overall, the readability is high, but not perfect.
Text 4012:
For example:

Score: 60%
Explanation: The text is wr

In [17]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

In [ ]:
download_file('/kaggle/working/TextReadabilityAnnotation/project-files/llm-data/llm-results/CLEAR_Corpus_6.01_LLM-output.csv', 'out')